In [1]:
%load_ext autoreload
%autoreload 2

In [45]:
from transformer.Transformer import build_transformer
from dataset import BilingualDataset,causal_mask
from config import get_config, get_weights_file_path, latest_weights_file_path

import torch
import torch.nn as nn 
from torch.utils.data import Dataset,DataLoader, random_split

import warnings
from tqdm import tqdm
import os
from pathlib import Path 

from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

In [46]:
config = get_config()

### Utility Functions

In [4]:
def get_all_sentences(ds,lang):
    for item in ds:
        yield item['translation'][lang]

In [5]:
def get_or_build_tokenizer(config,ds,lang):
    tokenizer_path = Path(config['tokenizer_file'].format(lang))
    if not Path.exists(tokenizer_path):
        tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
        tokenizer.pre_tokenizer = Whitespace()
        trainer = WordLevelTrainer(special_tokens=["[UNK]","[PAD]","[SOS]","[EOS]"],min_frequency=2)
        tokenizer.train_from_iterator(get_all_sentences(ds,lang),trainer=trainer)
        tokenizer.save(str(tokenizer_path))
    else:
        tokenizer = Tokenizer.from_file(str(tokenizer_path))
    
    return tokenizer

### Load Dataset

In [6]:
ds_raw = load_dataset(f"{config['datasource']}", 
                      f"{config['lang_src']}-{config['lang_tgt']}",
                      split='train',)

In [7]:
## Build tokenizers
tokenizer_src = get_or_build_tokenizer(config,ds_raw,config['lang_src'])
tokenizer_tgt = get_or_build_tokenizer(config,ds_raw,config['lang_tgt'])

In [8]:
train_ds_size = int(.9 * len(ds_raw))
val_ds_size = len(ds_raw) - train_ds_size
train_ds_raw, val_ds_raw = random_split(ds_raw,[train_ds_size,val_ds_size])

In [9]:
train_ds = BilingualDataset(train_ds_raw,
                            tokenizer_src,tokenizer_tgt,
                            config['lang_src'],config['lang_tgt'],
                            config['seq_len'])

val_ds = BilingualDataset(val_ds_raw,
                            tokenizer_src,tokenizer_tgt,
                            config['lang_src'],config['lang_tgt'],
                            config['seq_len'])

In [10]:
## Find max len of each sent in src & tgt
max_len_src = 0
max_len_tgt = 0

for item in ds_raw:
    src_ids = tokenizer_src.encode(item['translation'][config['lang_src']]).ids
    tgt_ids = tokenizer_tgt.encode(item['translation'][config['lang_tgt']]).ids
    max_len_src = max(max_len_src,len(src_ids))
    max_len_tgt = max(max_len_tgt,len(tgt_ids))

print(f'Max Length of src - {max_len_src}')
print(f'Max Length of tgt - {max_len_tgt}')

Max Length of src - 309
Max Length of tgt - 274


In [11]:
train_loader = DataLoader(train_ds,batch_size=config['batch_size'],shuffle=True)
val_loader = DataLoader(val_ds,batch_size=1,shuffle=True)

### Training Loop 

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

In [13]:
vocab_src_len = tokenizer_src.get_vocab_size()
vocab_tgt_len = tokenizer_tgt.get_vocab_size()
print(vocab_src_len,vocab_tgt_len)

15698 22463


In [47]:
model = build_transformer(vocab_src_len,vocab_tgt_len,
                          config['seq_len'],config['seq_len']).to(device)

In [15]:
sum([p.numel() for p in model.parameters() if p.requires_grad])

75165631

In [16]:
Path(f"{config['datasource']}_{config['model_folder']}").mkdir(parents=True, exist_ok=True)

In [17]:
optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], eps=1e-9)

In [18]:
initial_epoch = 0
global_step = 0
preload = config['preload']

In [19]:
model_filename = latest_weights_file_path(config) if preload == 'latest' else get_weights_file_path(config, preload) if preload else None

In [20]:
if model_filename:
    print(f'Preloading model {model_filename}')
    state = torch.load(model_filename)
    model.load_state_dict(state['model_state_dict'])
    initial_epoch = state['epoch'] + 1
    optimizer.load_state_dict(state['optimizer_state_dict'])
    global_step = state['global_step']
else:
    print('No model to preload, starting from scratch')

No model to preload, starting from scratch


In [21]:
loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id('[PAD]'), label_smoothing=0.1).to(device)

In [22]:
def greedy_decode(model, source, source_mask, tokenizer_src, tokenizer_tgt, max_len, device):
    sos_idx = tokenizer_tgt.token_to_id('[SOS]')
    eos_idx = tokenizer_tgt.token_to_id('[EOS]')

    # Precompute the encoder output and reuse it for every step
    encoder_output = model.encode(source, source_mask)
    # Initialize the decoder input with the sos token
    decoder_input = torch.empty(1, 1).fill_(sos_idx).type_as(source).to(device)
    while True:
        if decoder_input.size(1) == max_len:
            break

        # build mask for target
        decoder_mask = causal_mask(decoder_input.size(1)).type_as(source_mask).to(device)

        # calculate output
        out = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)

        # get next token
        prob = model.project(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        decoder_input = torch.cat(
            [decoder_input, torch.empty(1, 1).type_as(source).fill_(next_word.item()).to(device)], dim=1
        )

        if next_word == eos_idx:
            break

    return decoder_input.squeeze(0)


def run_validation(model, validation_ds, tokenizer_src, tokenizer_tgt, max_len, device, print_msg, global_step, writer, num_examples=2):
    model.eval()
    count = 0

    source_texts = []
    expected = []
    predicted = []

    try:
        # get the console window width
        with os.popen('stty size', 'r') as console:
            _, console_width = console.read().split()
            console_width = int(console_width)
    except:
        # If we can't get the console width, use 80 as default
        console_width = 80

    with torch.no_grad():
        for batch in validation_ds:
            count += 1
            encoder_input = batch["encoder_input"].to(device) # (b, seq_len)
            encoder_mask = batch["encoder_mask"].to(device) # (b, 1, 1, seq_len)

            # check that the batch size is 1
            assert encoder_input.size(
                0) == 1, "Batch size must be 1 for validation"

            model_out = greedy_decode(model, encoder_input, encoder_mask, tokenizer_src, tokenizer_tgt, max_len, device)

            source_text = batch["src_text"][0]
            target_text = batch["tgt_text"][0]
            model_out_text = tokenizer_tgt.decode(model_out.detach().cpu().numpy())

            source_texts.append(source_text)
            expected.append(target_text)
            predicted.append(model_out_text)
            
            # Print the source, target and model output
            print_msg('-'*console_width)
            print_msg(f"{f'SOURCE: ':>12}{source_text}")
            print_msg(f"{f'TARGET: ':>12}{target_text}")
            print_msg(f"{f'PREDICTED: ':>12}{model_out_text}")

            if count == num_examples:
                print_msg('-'*console_width)
                break

In [48]:
for epoch in range(initial_epoch, config['num_epochs']):
        torch.cuda.empty_cache()
        model.train()
        batch_iterator = tqdm(train_loader, desc=f"Processing Epoch {epoch:02d}")
        for idx, batch in enumerate(batch_iterator):
            #print(f'Epoch {epoch} and Batch - {idx}')
            encoder_input = batch['encoder_input'].to(device) # (b, seq_len)
            decoder_input = batch['decoder_input'].to(device) # (B, seq_len)
            encoder_mask = batch['encoder_mask'].to(device) # (B, 1, 1, seq_len)
            decoder_mask = batch['decoder_mask'].to(device) # (B, 1, seq_len, seq_len)

            # Run the tensors through the encoder, decoder and the projection layer
            encoder_output = model.encode(encoder_input, encoder_mask) # (B, seq_len, d_model)
            decoder_output = model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask) # (B, seq_len, d_model)
            proj_output = model.project(decoder_output) # (B, seq_len, vocab_size)

            # Compare the output with the label
            label = batch['label'].to(device) # (B, seq_len)

            # Compute the loss using a simple cross entropy
            loss = loss_fn(proj_output.view(-1, tokenizer_tgt.get_vocab_size()), label.view(-1))
            batch_iterator.set_postfix({"loss": f"{loss.item():6.3f}"})


            # Backpropagate the loss
            loss.backward()

            # Update the weights
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)

            global_step += 1

        # Run validation at the end of every epoch
        run_validation(model, val_loader, tokenizer_src, tokenizer_tgt, config['seq_len'], device, lambda msg: batch_iterator.write(msg), global_step, writer)

        # Save the model at the end of every epoch
        model_filename = get_weights_file_path(config, f"{epoch:02d}")
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'global_step': global_step
        }, model_filename)

Processing Epoch 00:   0%|          | 7/3638 [01:17<11:06:10, 11.01s/it, loss=10.057]


KeyboardInterrupt: 